# Loading import library

In [1]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from rdkit.Chem.Scaffolds.MurckoScaffold import MakeScaffoldGeneric
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles


from scripts import metrics
import importlib as imp
imp.reload(metrics)
import time

# Funktion for calculate metrics

In [2]:
def calculate_metrics(output_set, recall_set, number, scaffold_type, type_cluster, generator, receptor, save=True):

    mt = metrics.Metrics(type_cluster, scaffold_type, generator, number, receptor, save)     
    mt.load(output_set, recall_set)
    result = mt.calculate_metrics_with_return()
    display(result)
    return result

## Count metrics 
- Need the change parameters depend on which type of scaffold/cluster/generator/target you have

In [13]:
type_cluster = 'dis' #options: 'dis'|'sim' 
numbers = [0,1,2,3,4] #options: [0]|[0,1]|[0,1,2]|[0,1,2,3]|[0,1,2,3,4] -> all combinations 0-4, depends on which calculation you have
scaffold_type = 'csk' #options: 'csk'|'murcko'
generator = 'DrugEx_GT_desired' #options: 'Molpher'|'DrugEx'|'REINVENT'|'addcarbon'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'
save_options = True

In [14]:
for number in numbers:
    before = time.perf_counter()
    a0 = calculate_metrics(f"data/output_sets/{receptor}/cOS_{generator}_{type_cluster}_{number}_one_column.csv", \
                                                                f"data/input_recall_sets/{receptor}/cRS_{receptor}_{type_cluster}_{number}.csv",\
                                   number, scaffold_type, type_cluster,generator,receptor,save_options)
    a0
    after = time.perf_counter()
    print('Path found after', (after - before)/60, 'minutes.')

data/output_sets/Glucocorticoid_receptor/cOS_DrugEx_GT_desired_dis_0_one_column.csv


[13:52:50] Explicit valence for atom # 29 C, 5, is greater than permitted
[13:52:54] Explicit valence for atom # 4 C, 5, is greater than permitted
[13:52:56] Explicit valence for atom # 27 C, 5, is greater than permitted
[13:53:03] Explicit valence for atom # 16 C, 6, is greater than permitted
[13:53:04] Explicit valence for atom # 26 C, 5, is greater than permitted
[13:53:06] Explicit valence for atom # 34 C, 5, is greater than permitted
[13:53:11] Explicit valence for atom # 27 C, 5, is greater than permitted
[13:53:14] Explicit valence for atom # 18 C, 5, is greater than permitted
[13:53:16] Explicit valence for atom # 24 C, 5, is greater than permitted
[13:53:18] Explicit valence for atom # 20 C, 5, is greater than permitted
[13:53:22] Explicit valence for atom # 22 C, 5, is greater than permitted
[13:53:23] Explicit valence for atom # 12 C, 5, is greater than permitted
[13:53:24] Explicit valence for atom # 23 C, 5, is greater than permitted
[13:53:27] Explicit valence for atom # 

add_columns_same_like_input_function


/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

UASo:  28
SSr:  180
SSo:  452558
UASr:  40
TUPOR_1:  0.000278417352030016
TUPOR_2:  0.6700948828658425
Save


...

add_columns_same_like_input_function


/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

UASo:  36
SSr:  304
SSo:  427628
UASr:  43
TUPOR_1:  0.000595170634072083
TUPOR_2:  1.745884610929078
Save


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_GT_desired_4,dis,csk,126137,427628,36/43,0.837209,0.066373,0.000595,1.745885,0.294969,0.007731,3306


Path found after 4.128118185333308 minutes.


In [13]:
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'
save_options = True
for generator in ['DrugEx_GT_epsilon_0.1', 'DrugEx_GT_epsilon_0.6', 'DrugEx_RNN_epsilon_0.1', 'DrugEx_RNN_epsilon_0.6']:  #DrugEx_GT_1, DrugEx_RNN, REINVENT, GB_GA  ,'DrugEx_RNN','DrugEx_GT','DrugEx_GT_1','REINVENT','GB_GA', 'addcarbon'
    for scaffold_type in ['csk', 'murcko']:
        for type_cluster in ['dis', 'sim']:
            for number in [0,1,2,3,4]:
                before = time.perf_counter()
                a0 = calculate_metrics(f"data/output_sets/{receptor}/cOS_{generator}_{type_cluster}_{number}_one_column.csv", \
                                                                f"data/input_recall_sets/{receptor}/cRS_{receptor}_{type_cluster}_{number}.csv",\
                                   number, scaffold_type, type_cluster,generator,receptor,save_options)
                a0
                after = time.perf_counter()
                print('Path found after', (after - before)/60, 'minutes.')

data/output_sets/Glucocorticoid_receptor/cOS_DrugEx_GT_epsilon_0.1_dis_0_one_column.csv


[15:42:13] Explicit valence for atom # 3 C, 5, is greater than permitted
[15:42:29] Explicit valence for atom # 7 C, 5, is greater than permitted
[15:44:56] Explicit valence for atom # 3 C, 5, is greater than permitted
[15:45:01] Explicit valence for atom # 1 C, 5, is greater than permitted
[15:46:30] Explicit valence for atom # 14 C, 5, is greater than permitted


add_columns_same_like_input_function


/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

UASo:  22
SSr:  180
SSo:  781031
UASr:  40
TUPOR_1:  0.00012675553211076128
TUPOR_2:  0.4610574036113803
Save


...

add_columns_same_like_input_function


/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

UASo:  13
SSr:  42
SSo:  886542
UASr:  40
TUPOR_1:  1.539690166963325e-05
TUPOR_2:  0.14289287055774008
Save


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_RNN_epsilon_0.6_4,sim,murcko,371225,886542,13/40,0.325,0.008755,0.000015,0.142893,0.418734,0.000166,147


Path found after 2.5622297279500343 minutes.


## Calculate acarage value and save csv
- Do this step after calculate all metrics for each cluster

In [15]:
def avarage_value(numbers, type_cluster, scaffold_type, generator, receptor):

    path = f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/"
    link = {}
    for x in numbers:
        link[f'link_{x}'] = f'{path}metrics_cluster_{x}_{type_cluster}_{generator}.csv'

    df = pd.DataFrame()
    for x,y in link.items():
        a = pd.read_csv(y)
        df = pd.concat([df, a], ignore_index=True)
        

    a = df.mean(numeric_only=True)
    df.loc[len(df)] = [f"{generator}_mean", type_cluster, scaffold_type, a['USo'], a['SSo'], '-',\
                       a['TUPOR'],a['TUPOR_unique'],a['TUPOR_1'],a['TUPOR_2'], a['SESY'], a['ASER'], a['CwASo']]
    df = df.round(7)
    dff = df.copy()
    dff['SSo'] = dff['SSo'].apply(lambda x : "{:,}".format(x))
    dff['USo'] = dff['USo'].apply(lambda x : "{:,}".format(x))
    dff['CwASo'] = dff['CwASo'].apply(lambda x : "{:,}".format(x))


    display(df)

    mean = pd.DataFrame()
    mean = df.loc[len(numbers)].to_frame().T
                                        
    display(mean)
    dff.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean_with_coma.csv", index = False)
    df.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean.csv", index = False)
    dff.to_html(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean.html", index = False)
    mean.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}_mean_{scaffold_type}_{type_cluster}.csv", index=False) #float_format='%.7f'

    return mean

In [21]:
type_cluster = 'sim' #options: 'dis'|'sim' 
number = [0,1,2,3,4] #options: 1-5
scaffold_type = 'csk' #options: 'csk'|'murcko'
generator = 'DrugEx_GT_epsilon_0.1' #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'``

In [22]:
df = avarage_value(number, type_cluster, scaffold_type, generator, receptor)

,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_GT_epsilon_0.1_0,sim,csk,159254.0,822485.0,26/38,0.684211,0.042964,0.000404,1.694357,0.193625,0.008197,6742.0
1,DrugEx_GT_epsilon_0.1_1,sim,csk,113778.0,791168.0,26/38,0.684211,0.060136,0.000204,0.611094,0.143810,0.042221,33404.0
2,DrugEx_GT_epsilon_0.1_2,sim,csk,116779.0,881455.0,16/37,0.432432,0.037030,0.000066,0.209033,0.132484,0.002322,2047.0
3,DrugEx_GT_epsilon_0.1_3,sim,csk,142125.0,843014.0,31/35,0.885714,0.062319,0.000066,0.268783,0.168592,0.015515,13079.0
4,DrugEx_GT_epsilon_0.1_4,sim,csk,151498.0,810360.0,13/35,0.371429,0.024517,0.000019,0.083327,0.186951,0.000226,183.0
5,DrugEx_GT_epsilon_0.1_mean,sim,csk,136686.8,829696.4,-,0.611599,0.045393,0.000152,0.573319,0.165093,0.013696,11091.0


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
5,DrugEx_GT_epsilon_0.1_mean,sim,csk,136686.8,829696.4,-,0.611599,0.045393,0.000152,0.573319,0.165093,0.013696,11091.0


In [18]:
for generator in ['DrugEx_GT_epsilon_0.1_desired', 'DrugEx_GT_epsilon_0.6_desired', 'DrugEx_RNN_epsilon_0.1_desired', 'DrugEx_RNN_epsilon_0.6_desired', 'DrugEx_RNN_desired', 'DrugEx_GT_desired']:
    for scaffold_type in ['csk']:
        for type_cluster in ['dis']:
            number = [0,1,2,3,4]
            receptor = 'Glucocorticoid_receptor'
            avarage_value(number, type_cluster, scaffold_type, generator, receptor)

,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_GT_epsilon_0.1_desired_0,dis,csk,107798.0,663654.0,20/40,0.500000,0.046383,0.000136,0.365470,0.162431,0.005563,3692.0
1,DrugEx_GT_epsilon_0.1_desired_1,dis,csk,79974.0,612158.0,13/23,0.565217,0.070675,0.000058,0.202262,0.130643,0.000627,384.0
2,DrugEx_GT_epsilon_0.1_desired_2,dis,csk,97212.0,691712.0,9/40,0.225000,0.023145,0.000061,0.147038,0.140538,0.000130,90.0
3,DrugEx_GT_epsilon_0.1_desired_3,dis,csk,80623.0,646340.0,17/37,0.459460,0.056989,0.000163,0.354714,0.124738,0.005615,3629.0
4,DrugEx_GT_epsilon_0.1_desired_4,dis,csk,92283.0,524972.0,30/43,0.697674,0.075602,0.000404,0.867049,0.175787,0.001964,1031.0
5,DrugEx_GT_epsilon_0.1_desired_mean,dis,csk,91578.0,627767.2,-,0.489470,0.054559,0.000164,0.387306,0.146827,0.002780,1765.2


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
5,DrugEx_GT_epsilon_0.1_desired_mean,dis,csk,91578.0,627767.2,-,0.48947,0.054559,0.000164,0.387306,0.146827,0.00278,1765.2


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_GT_epsilon_0.6_desired_0,dis,csk,131092.0,531904.0,27/40,0.675000,0.051491,0.000228,0.748616,0.246458,0.007024,3736.0
1,DrugEx_GT_epsilon_0.6_desired_1,dis,csk,134840.0,535341.0,16/23,0.695652,0.051591,0.000082,0.479947,0.251877,0.006177,3307.0
2,DrugEx_GT_epsilon_0.6_desired_2,dis,csk,141333.0,559072.0,14/40,0.350000,0.024764,0.000116,0.411431,0.252799,0.000900,503.0
3,DrugEx_GT_epsilon_0.6_desired_3,dis,csk,127852.0,483275.0,23/37,0.621622,0.048620,0.000295,1.017825,0.264553,0.010764,5202.0
4,DrugEx_GT_epsilon_0.6_desired_4,dis,csk,129439.0,439692.0,36/43,0.837209,0.064680,0.000579,1.742432,0.294386,0.008033,3532.0
5,DrugEx_GT_epsilon_0.6_desired_mean,dis,csk,132911.2,509856.8,-,0.635897,0.048229,0.000260,0.880050,0.262015,0.006580,3256.0


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
5,DrugEx_GT_epsilon_0.6_desired_mean,dis,csk,132911.2,509856.8,-,0.635897,0.048229,0.00026,0.88005,0.262015,0.00658,3256.0


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_RNN_epsilon_0.1_desired_0,dis,csk,43852.0,540248.0,10/40,0.250000,0.057010,0.000083,0.091316,0.081170,0.000757,409.0
1,DrugEx_RNN_epsilon_0.1_desired_1,dis,csk,38017.0,400468.0,8/23,0.347826,0.091492,0.000055,0.090445,0.094931,0.005164,2068.0
2,DrugEx_RNN_epsilon_0.1_desired_2,dis,csk,55628.0,617489.0,2/40,0.050000,0.008988,0.000015,0.020945,0.090087,0.000006,4.0
3,DrugEx_RNN_epsilon_0.1_desired_3,dis,csk,41738.0,550408.0,8/37,0.216216,0.051803,0.000090,0.101477,0.075831,0.001019,561.0
4,DrugEx_RNN_epsilon_0.1_desired_4,dis,csk,67320.0,465521.0,27/43,0.627907,0.093272,0.000410,0.641956,0.144612,0.009473,4410.0
5,DrugEx_RNN_epsilon_0.1_desired_mean,dis,csk,49311.0,514826.8,-,0.298390,0.060513,0.000131,0.189228,0.097326,0.003284,1490.4


...

,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
5,DrugEx_RNN_epsilon_0.6_desired_mean,dis,csk,28218.8,147249.2,-,0.524203,0.184102,0.00073,0.577568,0.19261,0.008655,1281.8


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_RNN_desired_0,dis,csk,31024.0,114010.0,20/40,0.500000,0.161165,0.000789,0.612262,0.272116,0.003605,411.0
1,DrugEx_RNN_desired_1,dis,csk,34282.0,119130.0,17/23,0.739130,0.215603,0.000391,0.582611,0.287770,0.007177,855.0
2,DrugEx_RNN_desired_2,dis,csk,28627.0,106986.0,7/40,0.175000,0.061131,0.000304,0.217741,0.267577,0.001439,154.0
3,DrugEx_RNN_desired_3,dis,csk,31362.0,116933.0,22/37,0.594595,0.189591,0.001164,0.987010,0.268205,0.012717,1487.0
4,DrugEx_RNN_desired_4,dis,csk,33886.0,95047.0,32/43,0.744186,0.219615,0.002380,1.875722,0.356518,0.007933,754.0
5,DrugEx_RNN_desired_mean,dis,csk,31836.2,110421.2,-,0.550582,0.169421,0.001006,0.855069,0.290437,0.006574,732.2


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
5,DrugEx_RNN_desired_mean,dis,csk,31836.2,110421.2,-,0.550582,0.169421,0.001006,0.855069,0.290437,0.006574,732.2


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_GT_desired_0,dis,csk,96272.0,452558.0,28/40,0.700000,0.072711,0.000278,0.670095,0.212728,0.009864,4464.0
1,DrugEx_GT_desired_1,dis,csk,127604.0,494931.0,16/23,0.695652,0.054517,0.000088,0.491275,0.257822,0.004504,2229.0
2,DrugEx_GT_desired_2,dis,csk,115948.0,486904.0,12/40,0.300000,0.025874,0.000115,0.332196,0.238133,0.001310,638.0
3,DrugEx_GT_desired_3,dis,csk,124254.0,477020.0,27/37,0.729730,0.058729,0.000350,1.176440,0.260480,0.008893,4242.0
4,DrugEx_GT_desired_4,dis,csk,126137.0,427628.0,36/43,0.837209,0.066373,0.000595,1.745885,0.294969,0.007731,3306.0
5,DrugEx_GT_desired_mean,dis,csk,118043.0,467808.2,-,0.652518,0.055641,0.000285,0.883178,0.252826,0.006460,2975.8


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
5,DrugEx_GT_desired_mean,dis,csk,118043.0,467808.2,-,0.652518,0.055641,0.000285,0.883178,0.252826,0.00646,2975.8


In [37]:

df.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}_mean_{scaffold_type}_{type_cluster}.csv", index=False)

# Connect all mean value to one pandas DataFrame

In [24]:
def connect_mean_value(reeptor, scaffold_type, type_cluster, generators_name_list):
    path = f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/"
    link = {}
    for generator in generators_name_list:
        link[f'link_{generator}'] = f'{path}{generator}_mean_{scaffold_type}_{type_cluster}.csv'

    df = pd.DataFrame()
    for x,y in link.items():
        a = pd.read_csv(y)
        df = pd.concat([df, a], ignore_index=True)

    df = df.drop(columns = ['TUPOR_'])

    dff = df.copy()
    dff['SSo'] = dff['SSo'].apply(lambda x : "{:,}".format(x))
    dff['USo'] = dff['USo'].apply(lambda x : "{:,}".format(x))
    dff['CwASo'] = dff['CwASo'].apply(lambda x : "{:,}".format(x))

    
    dff.to_csv(f"{path}/mean_{scaffold_type}_{type_cluster}_with_coma.csv", index = False)
    df.to_csv(f"{path}/mean_{scaffold_type}_{type_cluster}.csv", index = False)
    dff.to_html(f"{path}/mean_{scaffold_type}_{type_cluster}.html", index = False)

    
    display(df)
    return df

In [25]:
type_cluster = 'dis' #options: 'dis'|'sim' 
scaffold_type = 'csk' #options: 'csk'|'murcko'
generators_name_list = [ 'DrugEx_GT', 'DrugEx_GT_epsilon_0.1','DrugEx_GT_epsilon_0.6', 'DrugEx_GT_desired', 'DrugEx_GT_epsilon_0.1_desired','DrugEx_GT_epsilon_0.6_desired'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'
df = connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)


,name,type_cluster,scaffold,USo,SSo,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_GT_mean,dis,csk,242118.8,891906.4,0.695865,0.028720,0.000154,1.010732,0.271447,0.008831,7851.4
1,DrugEx_GT_epsilon_0.1_mean,dis,csk,140428.0,809233.2,0.545466,0.039156,0.000130,0.511284,0.172969,0.004915,4063.2
2,DrugEx_GT_epsilon_0.6_mean,dis,csk,267648.0,907474.4,0.679649,0.025317,0.000147,1.055034,0.294816,0.007886,7186.4
3,DrugEx_GT_desired_mean,dis,csk,118043.0,467808.2,0.652518,0.055641,0.000285,0.883178,0.252826,0.006460,2975.8
4,DrugEx_GT_epsilon_0.1_desired_mean,dis,csk,91578.0,627767.2,0.489470,0.054559,0.000164,0.387306,0.146827,0.002780,1765.2
5,DrugEx_GT_epsilon_0.6_desired_mean,dis,csk,132911.2,509856.8,0.635897,0.048229,0.000260,0.880050,0.262015,0.006580,3256.0


In [21]:
type_cluster = 'dis' #options: 'dis'|'sim' 
scaffold_type = 'csk' #options: 'csk'|'murcko'
generators_name_list = [ 'DrugEx_RNN', 'DrugEx_RNN_epsilon_0.1','DrugEx_RNN_epsilon_0.6', 'DrugEx_RNN_desired', 'DrugEx_RNN_epsilon_0.1_desired','DrugEx_RNN_epsilon_0.6_desired'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'
df = connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)


,name,type_cluster,scaffold,USo,SSo,TUPOR,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_RNN_mean,dis,csk,110595.8,868587.2,0.665922,0.450804,0.127252,0.018142,16078.4
1,DrugEx_RNN_epsilon_0.1_mean,dis,csk,72000.6,797636.8,0.387368,0.220224,0.090468,0.005051,3941.0
2,DrugEx_RNN_epsilon_0.6_mean,dis,csk,87100.4,891065.4,0.620167,0.351509,0.097631,0.017469,15682.2
3,DrugEx_RNN_desired_mean,dis,csk,31836.2,110421.2,0.550582,0.855069,0.290437,0.006574,732.2
4,DrugEx_RNN_epsilon_0.1_desired_mean,dis,csk,49311.0,514826.8,0.298390,0.189228,0.097326,0.003284,1490.4
5,DrugEx_RNN_epsilon_0.6_desired_mean,dis,csk,28218.8,147249.2,0.524203,0.577568,0.192610,0.008655,1281.8


In [37]:
type_cluster = 'dis' #options: 'dis'|'sim' 
scaffold_type = 'csk' #options: 'csk'|'murcko'
generators_name_list = ['Molpher', 'DrugEx_GT', 'DrugEx_GT_epsilon_0.1','DrugEx_GT_epsilon_0.6','DrugEx_RNN','DrugEx_RNN_epsilon_0.1','DrugEx_RNN_epsilon_0.6','REINVENT', 'addcarbon','GB_GA'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

In [38]:
df = connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)

,name,type_cluster,scaffold,USo,SSo,TUPOR,TUPOR_2,SESY,ASER,CwASo
0,Molpher_mean,dis,csk,100068.0,823977.8,0.516682,0.360385,0.124815,0.004589,3041.2
1,DrugEx_GT_mean,dis,csk,242118.8,891906.4,0.695865,1.010732,0.271447,0.008831,7851.4
2,DrugEx_GT_epsilon_0.1_mean,dis,csk,140428.0,809233.2,0.545466,0.511284,0.172969,0.004915,4063.2
3,DrugEx_GT_epsilon_0.6_mean,dis,csk,267648.0,907474.4,0.679649,1.055034,0.294816,0.007886,7186.4
4,DrugEx_RNN_mean,dis,csk,110595.8,868587.2,0.665922,0.450804,0.127252,0.018142,16078.4
5,DrugEx_RNN_epsilon_0.1_mean,dis,csk,72000.6,797636.8,0.387368,0.220224,0.090468,0.005051,3941.0
6,DrugEx_RNN_epsilon_0.6_mean,dis,csk,87100.4,891065.4,0.620167,0.351509,0.097631,0.017469,15682.2
7,REINVENT_mean,dis,csk,169670.2,964580.2,0.559826,0.516835,0.175880,0.013492,13023.4
8,addcarbon_mean,dis,csk,856.8,1000000.0,0.033403,0.000148,0.000857,0.001827,1827.0
9,GB_GA_mean,dis,csk,2710.0,9998.0,0.197592,0.280268,0.271057,0.010821,108.2


In [25]:
type_cluster = 'dis' #options: 'dis'|'sim' 
scaffold_type = 'csk' #options: 'csk'|'murcko'
generators_name_list = ['DrugEx_epsilon_0.6', 'DrugEx_GT_1'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

df = connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)

,name,type_cluster,scaffold,USo,SSo,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_epsilon_0.6_mean,dis,csk,4889.8,9993.8,0.354260,0.710789,0.007132,0.948241,0.489278,0.008363,83.6
1,DrugEx_GT_1_mean,dis,csk,254849.2,908272.6,0.700922,0.027669,0.000153,1.025833,0.280542,0.009991,9050.0
